# Example stitching workflow

Using a small example dataset with two positions.

In [1]:
VISUALIZE_USING_NAPARI = False

In [2]:
import os, tempfile
import numpy as np
from pathlib import Path
from tqdm import tqdm
import dask.diagnostics

from ngff_stitcher import sample_data, msi_utils
# %matplotlib notebook

2023-09-26 11:17:24,376 [Geometry3D WARNING] 'Renderer' is deprecated, using 'Visualizer' instead.


In [3]:
# Start a dask cluster

from distributed import Client, LocalCluster

lc = LocalCluster(n_workers=1, threads_per_worker=None)
client = Client(lc)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62656,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:62661,Total threads: 8
Dashboard: http://127.0.0.1:62662/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:62659,


In [4]:
# Load dataset from file

from ngff_stitcher import io

filename = Path('/Users/malbert/software/ngff-stitcher/image-datasets/arthur_20220621_premovie_dish2-max.czi')
sims = io.read_mosaic_image_into_list_of_spatial_xarrays(filename, scene_index=0)

msims = [msi_utils.get_store_decorator(
    filename.with_suffix('.tile%03d.zarr' %isim), store_overwrite=False)(
        msi_utils.get_msim_from_sim)(sim, [])
    for isim, sim in tqdm(enumerate(sims))]

# msims = [msi_utils.get_msim_from_sim(sim, scale_factors=[]) for sim in sims]
print(f"Loaded {len(msims)} positions from {filename}. Transform key is {io.METADATA_TRANSFORM_KEY}.")

# Test with generated dataset

# sims = sample_data.generate_tiled_dataset(
#     ndim=2,
#     N_t=1,
#     N_c=1,
#     tile_size=1000,
#     tiles_x=5,
#     tiles_y=5,
#     zoom=4,
#     overlap=30,
#     drift_scale=0,
#     shift_scale=15,
# )


9it [00:00, 17.65it/s]

Loaded 9 positions from /Users/malbert/software/ngff-stitcher/image-datasets/arthur_20220621_premovie_dish2-max.czi. Transform key is affine_metadata.


In [5]:
if VISUALIZE_USING_NAPARI:

    from napari_stitcher import viewer_utils
    import importlib
    viewer_utils = importlib.reload(viewer_utils)
    
    import napari
    viewer = napari.Viewer()
    
    layer_tuples = viewer_utils.create_image_layer_tuples_from_msims(
        msims, n_colors=2, transform_key='affine_metadata')
    viewer_utils.add_image_layer_tuples_to_viewer(viewer, layer_tuples)
    viewer.reset_view()

In [6]:
from ngff_stitcher import registration, fusion, mv_graph

import importlib
registration = importlib.reload(registration)
mv_graph = importlib.reload(mv_graph)

print('Registering...')
with dask.diagnostics.ProgressBar():
    params = registration.register(
        msims[:], transform_key='affine_metadata', reg_channel_index=0)

for msim, param in zip(msims, params):
    msi_utils.set_affine_transform(msim, param, transform_key='affine_registered', base_transform_key='affine_metadata')

Registering...


In [7]:
import importlib
from ngff_stitcher import transformation
fusion = importlib.reload(fusion)
transformation = importlib.reload(transformation)

print('Fusing...')
# with dask.config.set(scheduler='single-threaded')
with dask.diagnostics.ProgressBar():
    fused = fusion.fuse(
        [msi_utils.get_sim_from_msim(msim) for msim in msims],
        transform_key='affine_registered')#.compute()

# mfused = msi_utils.get_msim_from_sim(fused, scale_factors=[])

Fusing...


In [ ]:
if VISUALIZE_USING_NAPARI:
    
    viewer = napari.Viewer(ndisplay=2)

    # lds = viewer_utils.create_image_layer_tuples_from_msims(
    #     msims, transform_key='affine_metadata', n_colors=2,
    #     name_prefix='pre-registered view',
    #     contrast_limits=(0, 1000),
    #     )
    # mlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_registered', n_colors=2,
        name_prefix='registered view',
        contrast_limits=(0, 1000),
        )
    mlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

    lds = viewer_utils.create_image_layer_tuples_from_msim(
        mfused,
        transform_key='affine_registered',
        name_prefix='fused',
        contrast_limits=[0, 1000])

    viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)



In [ ]:
with dask.diagnostics.ProgressBar():
    io.save_sim_as_tif('fused.tif', fused)